# People counter model preparation and compilation

## Install and import necessary Python libraries
In this first step, we install some useful Python libraries for us to easily prepare the model to be compiled by SageMaker NEO.

In [2]:
!pip3 install gluoncv
!pip3 install xtarfile

  Using cached gluoncv-0.10.4.post4-py2.py3-none-any.whl (1.3 MB)
  Using cached autocfg-0.0.8-py3-none-any.whl (13 kB)
  Using cached portalocker-2.3.2-py2.py3-none-any.whl (15 kB)
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.
  Created wheel for xtarfile: filename=xtarfile-0.1.0-py3-none-any.whl size=3861 sha256=dcdae3438d42b39375ebc83eceb8b91a15800b55d96943a64d75c261cc4b67ea
  Stored in directory: /home/ec2-user/.cache/pip/wheels/14/ed/96/b06b7600d9d2eb6068fed8ded4282aeb3addf5b213dfe48819
Successfully built xtarfile
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.


In [31]:
import numpy as np
import mxnet as mx 
import gluoncv as gcv
import xtarfile as tarfile
import sagemaker
import boto3
import time

## Model download and hybridize
Use the following code to download the pre-trained model from MXNet model zoo. If you want to use models from other frameworks, you can use check out the [model preparation page](https://docs.aws.amazon.com/sagemaker/latest/dg/neo-compilation-preparing-model.html) on developer's guide to find out more information.

In the second line, we hybridize the model and run through it once with randomized input of the expected input size. For more information about hybridizing models, please checkout the [MXNet explanation on hybridize](https://mxnet.apache.org/versions/1.7.0/api/python/docs/tutorials/packages/gluon/blocks/hybridize.html).

In [10]:
net = gcv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
net.hybridize()
out = net(mx.nd.ones((1,3,224,224)))

In [12]:
net.export('yolo3_mobilenet1.0_coco')
tar = tarfile.open("model.tar.gz", "w:gz")
for name in ["yolo3_mobilenet1.0_coco-0000.params", "yolo3_mobilenet1.0_coco-symbol.json"]:
    tar.add(name)
tar.close()

## Tar the model, and upload our model to S3
Since in this notebook instance, there is already a execution role assiciated with it, we do not need to provision all the credentials. And in this example, for the simplicity, we are going to get the default bucket. SageMaker has a default bucket for each account (starts with sagemaker-) in the same region of this notebook instance. We are going to store all of the artifacts and compiled models within that default bucket. Feel free to change the bucket location if needed otherwise.

In [32]:
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
print(role)
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base("MXNet-yolo3-mobilenet10-Jetson-Nano")
prefix = compilation_job_name + "/model"
model_path = sess.upload_data(path="model.tar.gz", key_prefix=prefix)

data_shape = '{"data":[1,3,224,224]}'
target_platform = {'Os': 'LINUX', 'Arch': 'ARM64', 'Accelerator': 'NVIDIA'}
compiler_options = '{"trt-ver": "7.1.3", "cuda-ver": "10.2", "gpu-code": "sm_53"}'
framework = "MXNET"
framework_version = "1.7"
compiled_model_path = "s3://{}/{}/output".format(bucket, compilation_job_name)

arn:aws:iam::121984884871:role/smem-role


## Start compilation job
Start compilation job, and start a polling process to wait for the compilation job to succeed. It could take around 5 minutes.

In [33]:
# Create a SageMaker client so you can submit a compilation job
sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': model_path,
        'DataInputConfig': data_shape,
        'Framework': framework.upper()
    },
    OutputConfig={
        'S3OutputLocation': compiled_model_path,
        'TargetPlatform': target_platform,
        'CompilerOptions': compiler_options
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 900
    }
)
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Done!


## Repackage compiled model in zip file for Greengrass deployment
Since NEO compilation output is in the format of .tar.gz, but AWS IoT Greengrass only accepts .zip. We need the following step to convert the compiled model to convert the format in order for the archived file to be downloaded and unpacked by Greengrass service.

In [38]:
s3_client = boto3.client('s3')
neo_compiled_model = 'compiled-model.tar.gz'
s3_client.download_file(bucket, prefix+'/model.tar.gz', neo_compiled_model)
!mkdir model
!tar zfxv compiled-model.tar.gz -C model/
!zip compiled-model.zip model/*
s3_client.upload_file('compiled-model.zip', bucket, '{}/model_zipped/model.zip'.format(compilation_job_name))

mkdir: cannot create directory ‘model’: File exists
yolo3_mobilenet1.0_coco-0000.params
yolo3_mobilenet1.0_coco-symbol.json
  adding: model/code/ (stored 0%)
  adding: model/disconnect/ (stored 0%)
  adding: model/model.tar.gz (deflated 2%)
  adding: model/server.log (deflated 84%)
  adding: model/yolo3_mobilenet1.0_coco-0000.params (deflated 8%)
  adding: model/yolo3_mobilenet1.0_coco-symbol.json (deflated 95%)


In [39]:
f's://{bucket}/{compilation_job_name}/model_zipped/model.zip'

's://sagemaker-us-west-2-121984884871/MXNet-yolo3-mobilenet10-Jetson-Nano-2021-11-15-03-18-21-004/model_zipped/model.zip'